In [1]:
import os
import pandas as pd
import pymysql

from tqdm.auto import tqdm

from constants import *

In [2]:
class umls_lookup():
    
    def __init__(self):
        
        self.db = pymysql.connect(host="localhost", user="umls_lookup", password="passw@rd", database="2022aa")
        self.cursor = self.db.cursor()
    
    def close(self):
        self.db.close()
    
    def get_rel(self, CUI1, CUI2):
        query = f"SELECT * from MRREL where CUI1='{CUI1}' and CUI2='{CUI2}'"
        self.cursor.execute(query)
        results = self.cursor.fetchall()
        cols = self.cursor.description
        list_cols = []
        for i in range(len(cols)):
            list_cols.append(cols[i][0])
        good_rel = []
        for row in results:
            # ID
            AUI1 = row[list_cols.index('AUI1')]
            if not AUI1:
                AUI1 = "        "
            AUI2 = row[list_cols.index('AUI2')]
            if not AUI2:
                AUI2 = "        "
            CUI1 = row[list_cols.index('CUI1')]
            if not CUI1:
                CUI1 = "        "
            CUI2 = row[list_cols.index('CUI2')]
            if not CUI2:
                CUI2 = "        "
            REL = row[list_cols.index('REL')]
        
            good_rel.append({"AUI1": AUI1, "CUI1": CUI1, "AUI2": AUI2, "CUI2": CUI2, "REL": REL})
        
        return good_rel

In [8]:
MAX_DIST = 20

with open(os.path.join(ROOT_PATH, "data", "2021AB_SN", "SRSTRE1")) as f:
    umls_relations_df = pd.read_csv(f, delimiter='|', names=["FirstTUI", "RelationTUI", "EndTUI"], index_col=False)


def get_UMLS_score(StartTUI, EndTUI, umls_relations_df):
    return len(umls_relations_df["RelationTUI"].loc[umls_relations_df["FirstTUI"]
                                                    == StartTUI].loc[umls_relations_df["EndTUI"] == EndTUI])

good_rel_full = []
umls_l = umls_lookup()

for filename in tqdm(os.listdir(ENTITIES_PATH)):
    
    with open(os.path.join(ENTITIES_PATH, filename), "r") as f:
        entities_df = pd.read_csv(f).drop("Unnamed: 0", axis=1)

    possible_relations_dicts = []

    for i in range(len(entities_df)):
        forward_df = entities_df.iloc[i + 1:].loc[entities_df["Sentence"] == entities_df.iloc[i]["Sentence"]]
        valid_relations = forward_df.loc[(forward_df["EndWord"] <= entities_df["StartWord"].iloc[i] + MAX_DIST) & (forward_df["Word"] != entities_df.iloc[i]["Word"])]
        for j in range(len(valid_relations)):
            if entities_df.iloc[i]["Word"] != entities_df.iloc[j]["Word"]:
                possible_relations_dicts.append({"First": i,
                                                 "End": i + j + 1,
                                                 "FirstWord": entities_df.iloc[i]["Word"],
                                                 "EndWord": valid_relations.iloc[j]["Word"],
                                                 "FirstCUI": entities_df.iloc[i]["CUI"],
                                                 "EndCUI": valid_relations.iloc[j]["CUI"],
                                                 "Distance": valid_relations.iloc[j]["StartWord"] - entities_df.iloc[i]["EndWord"],
                                                 "ScoreUMLS": get_UMLS_score(entities_df.iloc[i]["TUI"], entities_df.iloc[j]["TUI"], umls_relations_df)})
    
    for p_rel in possible_relations_dicts:
        possible_labels = []
        good_rel = umls_l.get_rel(p_rel["FirstCUI"], p_rel["EndCUI"])
        if good_rel:
            for rel in good_rel:
                possible_labels.append(rel['REL'])
            total_labels = list(set(possible_labels))
            for label in total_labels:
                new_good_rel = p_rel.copy()
                new_good_rel["Label"] = label
                good_rel_full.append(new_good_rel)

good_rel_df = pd.DataFrame(good_rel_full)
with open("relations_full.csv", "w") as f:
    good_rel_df.to_csv(f)

  0%|          | 0/701 [00:00<?, ?it/s]